## DSWD DROMIC Coronavirus disease (COVID-2019) situation reports

Reports from https://dromic.dswd.gov.ph/coronavirus-disease-covid-19-31-dec-2019/ in Tabular format.

In [338]:
import pandas as pd
import requests
import os
import glob
import re
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime
from decimal import Decimal
from docx import Document

In [339]:
# papermill parameters
output_folder = '../output/'

In [340]:
import os
if os.getcwd().endswith('notebooks'): root_path = '../'
else: root_path = ''
    
def path_of(path): 
    return root_path + path

In [341]:
region_psgc = [
    ['NCR', 130000000],
    ['REGION I', 10000000],
    ['REGION II', 20000000],
    ['REGION III', 30000000],
    ['CALABARZON', 40000000],
    ['MIMAROPA', 170000000],
    ['REGION V', 50000000],
    ['REGION VI', 60000000],
    ['REGION VII', 70000000],
    ['REGION VIII', 80000000],
    ['REGION IX', 90000000],
    ['REGION X', 100000000],
    ['REGION XI', 110000000],
    ['REGION XII', 120000000],
    ['CARAGA', 160000000],
    ['CAR', 140000000],
    ['ARMM', 150000000],
    ['BARMM', 150000000] # Renamed in source from sitrep 51
]
region_names = list(map(lambda x: x[0], region_psgc))

In [342]:
provdis_standard_psgc = [
    ['LANAO DEL NORTE', 103500000],
    ['MISAMIS OCCIDENTAL', 104200000],
    ['MISAMIS ORIENTAL', 104300000],
    ['DAVAO DEL NORTE', 112300000],
    ['DAVAO DEL SUR', 112400000],
    ['DAVAO ORIENTAL', 112500000],
    ['DAVAO DE ORO (COMPOSTELA VALLEY)', 118200000],
    ['DAVAO OCCIDENTAL', 118600000],
    ['COTABATO (NORTH COTABATO)', 124700000],
    ['SOUTH COTABATO', 126300000],
    ['SULTAN KUDARAT', 126500000],
    ['SARANGANI', 128000000],
    ['COTABATO CITY (Not a Province)', 129800000],
    ['NCR, CITY OF MANILA, FIRST DISTRICT (Not a Province)', 133900000],
    ['NCR, SECOND DISTRICT (Not a Province)', 137400000],
    ['NCR, THIRD DISTRICT (Not a Province)', 137500000],
    ['NCR, FOURTH DISTRICT (Not a Province)', 137600000],
    ['ABRA', 140100000],
    ['BENGUET', 141100000],
    ['IFUGAO', 142700000],
    ['KALINGA', 143200000],
    ['MOUNTAIN PROVINCE', 144400000],
    ['APAYAO', 148100000],
    ['BASILAN', 150700000],
    ['LANAO DEL SUR', 153600000],
    ['MAGUINDANAO', 153800000],
    ['SULU', 156600000],
    ['TAWI-TAWI', 157000000],
    ['AGUSAN DEL NORTE', 160200000],
    ['AGUSAN DEL SUR', 160300000],
    ['SURIGAO DEL NORTE', 166700000],
    ['SURIGAO DEL SUR', 166800000],
    ['DINAGAT ISLANDS', 168500000],
    ['MARINDUQUE', 174000000],
    ['OCCIDENTAL MINDORO', 175100000],
    ['ORIENTAL MINDORO', 175200000],
    ['PALAWAN', 175300000],
    ['ROMBLON', 175900000],
    ['CAMARINES SUR', 51700000],
    ['CATANDUANES', 52000000],
    ['MASBATE', 54100000],
    ['SORSOGON', 56200000],
    ['AKLAN', 60400000],
    ['ANTIQUE', 60600000],
    ['CAPIZ', 61900000],
    ['ILOILO', 63000000],
    ['NEGROS OCCIDENTAL', 64500000],
    ['GUIMARAS', 67900000],
    ['BOHOL', 71200000],
    ['CEBU', 72200000],
    ['NEGROS ORIENTAL', 74600000],
    ['SIQUIJOR', 76100000],
    ['EASTERN SAMAR', 82600000],
    ['LEYTE', 83700000],
    ['NORTHERN SAMAR', 84800000],
    ['SAMAR (WESTERN SAMAR)', 86000000],
    ['SOUTHERN LEYTE', 86400000],
    ['BILIRAN', 87800000],
    ['ZAMBOANGA DEL NORTE', 97200000],
    ['ZAMBOANGA DEL SUR', 97300000],
    ['ZAMBOANGA SIBUGAY', 98300000],
    ['CITY OF ISABELA (Not a Province)', 99700000],
    ['BUKIDNON', 101300000],
    ['CAMIGUIN', 101800000],
    ['ILOCOS NORTE', 12800000],
    ['ILOCOS SUR', 12900000],
    ['LA UNION', 13300000],
    ['PANGASINAN', 15500000],
    ['BATANES', 20900000],
    ['CAGAYAN', 21500000],
    ['ISABELA', 23100000],
    ['NUEVA VIZCAYA', 25000000],
    ['QUIRINO', 25700000],
    ['BATAAN', 30800000],
    ['BULACAN', 31400000],
    ['NUEVA ECIJA', 34900000],
    ['PAMPANGA', 35400000],
    ['TARLAC', 36900000],
    ['ZAMBALES', 37100000],
    ['AURORA', 37700000],
    ['BATANGAS', 41000000],
    ['CAVITE', 42100000],
    ['LAGUNA', 43400000],
    ['QUEZON', 45600000],
    ['RIZAL', 45800000],
    ['ALBAY', 50500000],
    ['CAMARINES NORTE', 51600000]
]
provdis_dswd_psgc = [
    ['CALOOCAN CITY', 137500000],
    ['LAS PINAS', 137600000],
    ['MAKATI CITY', 137600000],
    ['MALABON CITY', 137500000],
    ['MANDALUYONG CITY', 137400000],
    ['MANILA CITY', 133900000],
    ['MARIKINA CITY', 137400000],
    ['MUNTINLUPA CITY', 137600000],
    ['NAVOTAS', 137500000],
    ['PARANAQUE CITY', 137600000],
    ['PASAY CITY', 137600000],
    ['PASIG CITY', 137400000],
    ['PATEROS', 137600000],
    ['TAGUIG CITY', 137600000],
    ['QUEZON CITY', 137400000],
    ['SAN JUAN CITY', 137400000],
    ['VALENZUELA CITY', 137500000],
    ['WESTERN SAMAR', 86000000],
    ['BASILAN (ISABELA CITY)', 99700000],
    ['DAVAO DE ORO', 118200000],
    ['NORTH COTABATO', 124700000]
]
provdis_psgc = provdis_standard_psgc + provdis_dswd_psgc
provdis_names = list(map(lambda x: x[0], provdis_psgc))

In [343]:
geocode_psgc_df = pd.read_csv(path_of('datasets/geocode/psgc.csv'))
geocode_dswd_df = pd.read_csv(path_of('datasets/geocode/dswd.csv'))
geocode_df = geocode_psgc_df.append(geocode_dswd_df, ignore_index=True)
geocode_df['SMC_NAME'] = geocode_df['SMC_NAME'].str.upper()
print('Geocode count %s' % len(geocode_df))

Geocode count 1697


In [344]:
def process(doc, sitrep_no, timestamp):
    document = Document(doc)
    tables = document.tables
    table = None
    for tbl in tables:
        header = tbl.rows[0].cells[1].paragraphs[0].text.upper().strip()
        if header == 'REGION / PROVINCE / MUNICIPALITY':
            table = tbl
            break
    if table is None: # No data available
        return

    data_start_row = 3
    if table.rows[3].cells[0].paragraphs[0].text.upper().strip() == 'REGION / PROVINCE / MUNICIPALITY': # Start at index 5 because row is still header (SitRep 28)
        data_start_row = 5
    data = []
    region = None
    reg_psgc = None
    province = None
    for i, row in enumerate(table.rows):
        if i >= data_start_row:
            cur_row = []
            cur_row.append(sitrep_no)
            cur_row.append(timestamp)
            for j, cell in enumerate(row.cells):
                p = cell.paragraphs[0] # get the first paragraph of the cell only
                if j == 0: # Parent/Child
                    geographic_level = 'MUNICITY'
                    is_child = p.text == '\xa0' or p.text.strip() == ''
                    is_plgu = False
                    parent = p.text.strip().upper()
                    try: r_idx = region_names.index(parent) 
                    except: r_idx = -1
                    if (not is_child) and (r_idx >= 0):
                        print('Processing Region %s' % parent)
                        geographic_level = 'REGION'
                        region = parent
                        reg_psgc = region_psgc[r_idx][1]
                        geographic_level
                        province = None
                        prv_psgc = None
                    elif parent.startswith('PLGU'):
                        print('Processing Province (PLGU) %s' % parent)
                        geographic_level = 'PROVINCE'
                        is_plgu = True
                        parent_plgu = parent[5:]
                        try: p_idx = provdis_names.index(parent_plgu) 
                        except: p_idx = -1
                        if p_idx >=0: 
                            province = parent_plgu
                            prv_psgc = provdis_psgc[p_idx][1]
                        else: raise RuntimeError('Region or Province "%s" not found' % parent)
                    elif not is_child:
                        if region == 'NCR':
                            geographic_level = 'MUNICITY'
                        else:
                            geographic_level = 'PROVINCE'
                            print('Processing Province %s' % parent)
                        try: p_idx = provdis_names.index(parent) 
                        except: p_idx = -1
                        if p_idx >=0: 
                            province = parent
                            prv_psgc = provdis_psgc[p_idx][1]
                        else: raise RuntimeError('Region or Province "%s" not found' % parent)
                    cur_row.append(geographic_level)
                    cur_row.append(is_child)
                    cur_row.append(is_plgu)
                    cur_row.append(reg_psgc)
                    cur_row.append(region)
                    cur_row.append(prv_psgc)
                    cur_row.append(province)
                elif j == 1: # Location
                    location = p.text.upper().strip()
                    if location == '' and province == 'LAGUNA' and sitrep_no == 38: # The Laguna PLGU label was not present in sitrep 38
                        location = 'PLGU LAGUNA'
                        cur_row[2] = 'PROVINCE'
                        cur_row[4] = True
                    elif location == 'COTABATO CITY': # Cotabato City is in a different province now in PSGC not in Sultan Kudarat
                        cur_row[7] = 129800000
                        cur_row[8] = 'COTABATO CITY (Not a Province)'
                    elif location.startswith('PLGU'): # Sometimes PLGU is at the municipality level
                        cur_row[2] = 'PROVINCE'
                        cur_row[4] = True
                    cur_row.append(location)
                else: # Numerical data
                    if p.text.strip() == '-' or p.text.strip() == '':
                        cur_row.append(None)
                    else:
                        num_val = re.sub('[^0-9\.]+', '', p.text)
                        num_val = re.sub('[.](?=.*[.])', '', num_val) # remove multiple '.' characters as in sitrep 46 error: Unable to parse Decimal "540.000.00"
                        try: cur_row.append(Decimal(num_val)) # remove commas and double
                        except: raise RuntimeError('Unable to parse Decimal "%s"' % num_val)
            data.append(cur_row)


    data_df = pd.DataFrame(data, columns=['sitrep', 'timestamp', 'geo_level', 'is_child', 'is_plgu', 'reg_psgc', 'region', 'prv_psgc', 'province', 'location', 'dswd', 'lgu', 'ngo', 'others', 'total'])
    
    data_df = data_df.fillna(0) # Workaround for merging prv_psgc
    data_df['reg_psgc'] = data_df['reg_psgc'].astype('int64')
    data_df['prv_psgc'] = data_df['prv_psgc'].astype('int64')
    geocoded_df = pd.merge(data_df, geocode_df, how='left', left_on=['reg_psgc', 'prv_psgc', 'location'], right_on=['REG_PSGC', 'PRVDIS_PSGC', 'SMC_NAME'])
    geocoded_df = geocoded_df.rename(columns={
        'PSGC': 'psgc_code', 
        'location': 'dswd_name',
        'SMC_NAME': 'psgc_name',
        'region': 'dswd_region',
        'REG_NAME': 'psgc_region',
        'REG_PSGC': 'psgc_region_code',
        'PRVDIS_NAME': 'psgc_province',
        'PRVDIS_PSGC': 'psgc_province_code',
        'province': 'dswd_province',
        'CITY_CLASS': 'city_class',
        'INCOME_CLASS': 'income_class',
        'LONGITUDE': 'longitude',
        'LATITUDE': 'latitude',
        'dswd': 'coa_dswd',
        'lgu': 'coa_lgu',
        'ngo': 'coa_ngo',
        'others': 'coa_others',
        'total': 'coa_total'
    })
    # Join with original PSGC names to get psgc_name
    geocode_psgc_part_df = geocode_psgc_df[['SMC_NAME', 'PSGC', 'REG_PSGC', 'PRVDIS_PSGC']]
    geocoded_df = geocoded_df.merge(geocode_psgc_part_df, how='left', left_on=['psgc_code', 'psgc_province_code', 'psgc_region_code'], right_on=['PSGC', 'PRVDIS_PSGC', 'REG_PSGC'])
    geocoded_df['psgc_name'] = geocoded_df['SMC_NAME']
    
    geocoded_df = geocoded_df[['timestamp', 'sitrep', 'geo_level', 'is_plgu', 'dswd_name', 'psgc_code', 'psgc_name', 'dswd_province', 'psgc_province', 'psgc_province_code', 'dswd_region', 'psgc_region', 'psgc_region_code', 'latitude', 'longitude', 'coa_dswd', 'coa_lgu', 'coa_ngo', 'coa_others', 'coa_total']]
    geocoded_df.loc[(geocoded_df['dswd_province'] == 0), 'dswd_province'] = None # Replace 0 dswd_province with no data
    
    with_psgc = geocoded_df['psgc_name'].str.len() > 0 
    success_df = geocoded_df[with_psgc]
    without_psgc = (~geocoded_df['is_plgu']) & geocoded_df['psgc_name'].isna() & ((geocoded_df['geo_level'] == 'MUNICITY') | ((geocoded_df['geo_level'] == 'PROVINCE') & (geocoded_df['dswd_region'] == 'NCR')))
    failed_df = geocoded_df[without_psgc]
    print('Records: %s, Success: %s, Failed: %s' % (len(data), len(success_df), len(failed_df)))
    if len(failed_df) > 0:
        failed_log_df = failed_df[['dswd_name', 'dswd_province', 'dswd_region']]
        failed_log_df.to_csv('{}/failed_locs_sr{:0>3d}.csv'.format(output_folder, sitrep_no), index = False)
    geocoded_df.to_csv('{}/dromic_covid19_sitreps_coa_sr{:0>3d}_coa.csv'.format(output_folder, sitrep_no), index = False)

In [345]:
inputs = []
for input_file in glob.glob(path_of('datasets/sitreps/*.docx')):
    fparts = os.path.splitext(os.path.basename(input_file))[0].split('-')
    sitrep_no = fparts[3]
    if fparts[8] == 'COVID':
        dom = fparts[12][:2]
        mon = fparts[13]
        yr = fparts[14][:4]
        hr = fparts[15][:3]
    else: 
        dom = fparts[11][:2]
        mon = fparts[12]
        yr = fparts[13][:4]
        hr = fparts[14][:3]
    
    datetime_string = '%s-%s-%s-%s' % (yr, mon, dom, hr)
    timestamp = datetime.strptime(datetime_string, '%Y-%B-%d-%I%p')
    inputs.append([int(sitrep_no), timestamp, input_file])

    
min_sitrep = 63 # Process only sitreps from this value for incremental processing

inputs.sort(key=lambda r: r[0]) # reverse=True for testing to get most recent file
for inp in inputs:
    sitrep_no = inp[0]
    timestamp = inp[1]
    doc = inp[2]
    if sitrep_no >= min_sitrep:
        print('Proecssing SR%s t=%s %s' % (inp[0], inp[1], inp[2]))
        process(doc, sitrep_no, timestamp)

Proecssing SR1 t=2020-03-20 20:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-1-on-the-Coronavirus-Disease-COVID-19-as-of-20-March-2020-8PM-2.docx
Proecssing SR2 t=2020-03-21 06:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-2-on-the-Coronavirus-Disease-COVID-19-as-of-21-March-2020-6AM.docx
Proecssing SR3 t=2020-03-21 19:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-3-on-the-Coronavirus-Disease-COVID-19-as-of-21-March-2020-7PM-1.docx
Proecssing SR4 t=2020-03-22 06:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-4-on-the-Coronavirus-Disease-COVID-19-as-of-22-March-2020-6AM.docx
Proecssing SR5 t=2020-03-22 18:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-5-on-the-Coronavirus-Disease-COVID-19-as-of-22-March-2020-6PM.docx
Proecssing SR6 t=2020-03-23 06:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-6-on-the-Coronavirus-Disease-COVID-19-as-of-23-March-2020-6AM.docx
Proecssing SR7 t=2020-03-23 18:00:00 ../datasets/sitreps/DSWD-DROMIC-Report-7-on-the-Coronavirus-Disease-COVID-19-as-of-23-March-2020-6PM.